In [2]:
import random
random.seed()
NIC = ["A", "B", "C"]
NIC_LOCATION = [1, 25, 60]
NO_NIC = 3
ERROR = "X"
EMPTY = "-"
ETH_LEN = 64

In [4]:
instructions = open("instructions.txt", "r")
log = open("log.txt", "w")
ethernet = [EMPTY for i in range(ETH_LEN)]
packageDirection = ["N" for i in range(ETH_LEN)] # N - none, L - left, R - rigt, B - both
transferQ = [0 for i in range(NO_NIC)]
waitQ = [0 for i in range(NO_NIC)]
isTransmitting = [False for i in range(NO_NIC)]
attempts = [0 for i in range(NO_NIC)]
event = instructions.read(1)
packetsLeft = 0
while(len(event) == 1 or packetsLeft > 0):
    if(len(event) == 0):
        event = "-"
    packetsLeft = 0
    newEthernet = [EMPTY for i in range(ETH_LEN)]
    newPackageDirection = ["N" for i in range(ETH_LEN)]
    if(event != "-"):
        event = int(event)
        transferQ[event] = ETH_LEN * 2
    for i in range(NO_NIC):
        if(transferQ[i] > 0 and waitQ[i] == 0 and (ethernet[NIC_LOCATION[i]] == EMPTY or ethernet[NIC_LOCATION[i]] == NIC[i])):
            newEthernet[NIC_LOCATION[i]] = NIC[i]
            newPackageDirection[NIC_LOCATION[i]] = "B"
            transferQ[i] -= 1
            if(transferQ[i] == 0):
                isTransmitting[i] = False
                attempts[i] = 0
            elif(not isTransmitting[i]):
                attempts[i] += 1
                isTransmitting[i] = True
                if(attempts[i] > 16):
                    attempts[i] = 0
                    isTransmitting[i] = False
                    transferQ[i] = 0
        if(waitQ[i] > 0):
            waitQ[i] -= 1
        if(isTransmitting[i] and ethernet[NIC_LOCATION[i]] == ERROR):
            isTransmitting[i] = False
            waitQ[i] = random.randint(0, pow(2, min(attempts[i], 10)) - 1) * ETH_LEN
            transferQ[i] = ETH_LEN * 2
        packetsLeft += transferQ[i]
    for i in range(ETH_LEN):
        c = ethernet[i]
        if(c != EMPTY):
            d = packageDirection[i]
            if(d == "L" and i - 1 >= 0):
                if(newEthernet[i - 1] != EMPTY):
                    newEthernet[i - 1] = ERROR
                    if(newPackageDirection[i - 1] == "L" or newPackageDirection[i - 1] == "N"):
                        newPackageDirection[i - 1] = "L"
                    else:
                        newPackageDirection[i - 1] = "B"
                else:
                    newEthernet[i - 1] = c
                    newPackageDirection[i - 1] = d
            if(d == "P" and i + 1 < ETH_LEN):
                if(newEthernet[i + 1] != EMPTY):
                    newEthernet[i + 1] =  ERROR
                    if(newPackageDirection[i + 1] == "P" or newPackageDirection[i + 1] == "N"):
                        newPackageDirection[i + 1] = "P"
                    else:
                        newPackageDirection[i + 1] = "B"
                else:
                    newEthernet[i + 1] = c
                    newPackageDirection[i + 1] = d
            if(d == "B"):
                if(i - 1 >= 0):
                    if(newEthernet[i - 1] != EMPTY):
                        newEthernet[i - 1] = ERROR
                        if(newPackageDirection[i - 1] == "L" or newPackageDirection[i - 1] == "N"):
                            newPackageDirection[i - 1] = "L"
                        else:
                            newPackageDirection[i - 1] = "B"
                    else:
                        newEthernet[i - 1] = c
                        newPackageDirection[i - 1] = "L"
                if(i + 1 < ETH_LEN):
                    if(newEthernet[i + 1] != EMPTY):
                        newEthernet[i + 1] = ERROR
                        if(newPackageDirection[i + 1] == "P" or newPackageDirection[i + 1] == "N"):
                            newPackageDirection[i + 1] = "P"
                        else:
                            newPackageDirection[i + 1] = "B"
                    else:
                        newEthernet[i + 1] = c
                        newPackageDirection[i + 1] = "P"
            # if(i > 0 and ethernet[i - 1] != EMPTY and (d == "B" or d == "L") and (packageDirection[i - 1] == "B" or packageDirection[i - 1] == "R")):
            #     newEthernet[i - 1] = ERROR
            #     newPackageDirection[i - 1] = "L"
            #     newEthernet[i] = ERROR
            #     newPackageDirection[i] = "R"
    ethernet = newEthernet
    packageDirection = newPackageDirection
    for u in ethernet:
        if u != EMPTY:
            packetsLeft += 1

    
                    
                    
            
            




    # Wypisanie 
    line = ""
    for c in ethernet:
        log.write(c)
        line += c
    # print(line)
    log.write("\n")
    event = instructions.read(1)
instructions.close()
log.close()